In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

/home/vi/.dgl/cora_v2.zip:   0%|          | 0.00/132k [00:00<?, ?B/s]

Extracting file to /home/vi/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [3]:
g = dataset[0]

In [4]:
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [5]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [6]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )


model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.122 (best 0.122), test acc: 0.130 (best 0.130)
In epoch 5, loss: 1.879, val acc: 0.520 (best 0.520), test acc: 0.550 (best 0.550)
In epoch 10, loss: 1.785, val acc: 0.532 (best 0.532), test acc: 0.584 (best 0.577)
In epoch 15, loss: 1.667, val acc: 0.620 (best 0.620), test acc: 0.648 (best 0.648)
In epoch 20, loss: 1.527, val acc: 0.674 (best 0.674), test acc: 0.678 (best 0.678)
In epoch 25, loss: 1.368, val acc: 0.718 (best 0.718), test acc: 0.718 (best 0.718)
In epoch 30, loss: 1.197, val acc: 0.732 (best 0.732), test acc: 0.737 (best 0.737)
In epoch 35, loss: 1.022, val acc: 0.744 (best 0.744), test acc: 0.760 (best 0.760)
In epoch 40, loss: 0.853, val acc: 0.750 (best 0.750), test acc: 0.767 (best 0.767)
In epoch 45, loss: 0.699, val acc: 0.750 (best 0.752), test acc: 0.775 (best 0.773)
In epoch 50, loss: 0.566, val acc: 0.754 (best 0.754), test acc: 0.782 (best 0.778)
In epoch 55, loss: 0.456, val acc: 0.756 (best 0.756), test acc: 0.779 (best 0

In [7]:
g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model)

In epoch 0, loss: 1.945, val acc: 0.214 (best 0.214), test acc: 0.212 (best 0.212)
In epoch 5, loss: 1.892, val acc: 0.550 (best 0.550), test acc: 0.564 (best 0.564)
In epoch 10, loss: 1.812, val acc: 0.560 (best 0.560), test acc: 0.602 (best 0.602)
In epoch 15, loss: 1.708, val acc: 0.692 (best 0.692), test acc: 0.706 (best 0.706)
In epoch 20, loss: 1.581, val acc: 0.726 (best 0.726), test acc: 0.728 (best 0.728)
In epoch 25, loss: 1.435, val acc: 0.724 (best 0.726), test acc: 0.737 (best 0.728)
In epoch 30, loss: 1.274, val acc: 0.726 (best 0.726), test acc: 0.746 (best 0.728)
In epoch 35, loss: 1.107, val acc: 0.724 (best 0.726), test acc: 0.752 (best 0.728)
In epoch 40, loss: 0.942, val acc: 0.726 (best 0.726), test acc: 0.755 (best 0.728)
In epoch 45, loss: 0.787, val acc: 0.736 (best 0.736), test acc: 0.761 (best 0.761)
In epoch 50, loss: 0.647, val acc: 0.742 (best 0.742), test acc: 0.763 (best 0.760)
In epoch 55, loss: 0.527, val acc: 0.756 (best 0.756), test acc: 0.768 (best 0